In [1]:
from tqdm import tqdm_notebook as tqdm
from cfb import models as cfb_models
import pandas as pd; DF=pd.DataFrame
import numpy as np; import requests
import os
from django_pandas.io import read_frame as RF
from os.path import join
pwd=os.getcwd()
from utils import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Create Season Instances
for year in range(1900,2100):
    season,c = Season.objects.get_or_create(
        year=year,defaults=dict(year=year)
    )

## Create Season/Coach/Conference Instances

In [20]:
path = join(pwd,'z_data\Coaches9.csv')
coaches_df=pd.read_csv(path)
col_list=['SchoolPay','TotalPay','Bonus','BonusPaid','AssistantPay','Buyout']
coaches_df = DataFrameCleaner(coaches_df).keep_numeric_only(col_list=col_list)


### Create Season Objects 
for year in range(1900,2100):
    season,c = Season.objects.get_or_create(
        year=year,defaults=dict(year=year)
    )

###  Create TeamSeason Objects for 2019
season = Season.objects.get(year=2019)
def main():
    for tup in tqdm(coaches_df.itertuples()):
        # get or create school
        school,c=cfb_models.School.objects.get_or_create(
            name=tup.School,defaults=dict(name=tup.School)
        )
        # get or create team
        team,c=cfb_models.Team.objects.get_or_create(
            school=school,type='Football',
            defaults=dict(school=school,type='Football')
        )
        # get or create conference
        conf,c=cfb_models.Conference.objects.get_or_create(
            name=tup.Conference,defaults=dict(name=tup.Conference)
        )

        # get or create coach
        coach_name_arr = tup.Coach.strip().split(' ')
        cfn=coach_name_arr[0]; cln=coach_name_arr[1]
        coach,c = cfb_models.Coach.objects.get_or_create(
            firstName=cfn,lastName=cln,
            defaults=dict(firstName=cfn,lastName=cln)
        )

        # get or create team season
        tseason,c = TeamSeason.objects.update_or_create(
            team=team,season=season,conference=conf,coach=coach,
            defaults=dict(team=team,season=season,conference=conf,coach=coach)
        )
        print(tseason.coach)
        cpay,c = CoachPay.objects.update_or_create(
            teamseason=tseason,base=tup.SchoolPay,total=tup.TotalPay,
            bonus=tup.Bonus,bonusPaid=tup.BonusPaid,buyout=tup.Buyout,
            defaults=dict(
                teamseason=tseason,base=tup.SchoolPay,total=tup.TotalPay,
                bonus=tup.Bonus,bonusPaid=tup.BonusPaid,buyout=tup.Buyout,        
            )
        )

main()  

Troy Calhoun
Terry Bowden
Nick Saban
Bill Clark
Scott Satterfield
Kevin Sumlin
Herm Edwards
Chad Morris
Blake Anderson
Jeff Monken
Gus Malzahn
Mike Neu
Matt Rhule
Bryan Harsin
Steve Addazio
Mike Jinks
Kalani Sitake
Lance Leipold
Justin Wilcox
Josh Heupel
John Bonamego
Brad Lambert
Luke Fickell
Dabo Swinney
Joe Moglia
Mike MacIntyre
Mike Bobo
Randy Edsall
David Cutcliffe
Scottie Montgomery
Chris Creighton
Dan Mullen
Lane Kiffin
Butch Davis
Willie Taggart
Jeff Tedford
Kirby Smart
Chad Lunsford
Shawn Elliott
Paul Johnson
Nick Rolovich
Major Applewhite
Lovie Smith
Tom Allen
Kirk Ferentz
Matt Campbell
David Beaty
Bill Snyder
Sean Lewis
Mark Stoops
Turner Gill
Skip Holtz
Billy Napier
Matt Viator
Bobby Petrino
Ed Orgeron
Doc Holliday
DJ Durkin
Mark Whipple
Mike Norvell
Mark Richt
Chuck Martin
Jim Harbaugh
Mark Dantonio
Rick Stockstill
P.J. Fleck
Matt Luke
Joe Moorhead
Barry Odom
Ken Niumatalolo
Scott Frost
Jay Norvell
Tony Sanchez
Bob Davie
Doug Martin
Larry Fedora
Dave Doeren
Seth Littrell
R

## Create Team Season instances for previous seasons

In [5]:
def create_team_season_instances():
    for season in tqdm(Season.objects.all()):
        for team in Team.objects.all():
            teamseason,c = TeamSeason.objects.get_or_create(
                team=team,season=season,
                defaults=dict(team=team,season=season),
            )
            
#create_team_season_instances()

### Scrape cfbstats.com to extract url id for each team

In [34]:
#### Get Teams URL ID from cfbstats

def translate_team_name(old_name):
    team_match_dict={
        'BYU':'Brigham Young',"Hawai'i":'Hawaii', 
        'Miami (Florida)':'Miami (Fla.)', 'SMU':'Southern Methodist', 
        'TCU':'Texas Christian', 'UAB':'Alabama at Birmingham', 
        'UCF':'Central Florida', 'UNLV':'Nevada-Las Vegas',
        'USC':'Southern California','UTEP':'Texas-El Paso', 
        'UTSA':'Texas-San Antonio','Appalachian St.':'Appalachian State',
        'Fla. Atlantic':'Florida Atlantic',"Florida Int'l":'Florida International',
        'Western Ky.':'Western Kentucky','Middle Tenn. St.':'Middle Tennessee'
    } 
    old_name = replace_abbrevs(old_name)
    if old_name in team_match_dict.keys():
        new_name = team_match_dict[old_name]    
    else:
        new_name = old_name
    return new_name

def replace_abbrevs(old_name):
    new_name=old_name
    replace_arr=[
        ['St.','State'],
        ['Ala.','Alabama'],
        ['Ga.','Georgia'],
        ['Fla.','Florida'],
        ['Ill.','Illinois'],
        ['Caro.','Carolina'],
        ['Ky.','Kentucky'],
        ['La.','Louisiana'],
        ['Mich.','Michigan'],
        ['Miss.','Mississippi'],
    ]
    for r in replace_arr:
        new_name=new_name.replace(r[0],r[1])
    return new_name
    
def get_team_home_page_dfs_from_cfbstats(team_id,year):
    url = f'http://www.cfbstats.com/{year}/team/{team_id}/index.html'
    r=requests.get(url)
    try:
        dfs=pd.read_html(r.content)
    except ValueError:
        dfs=[]
    return dfs

def main():
    found_count=0
    target = len(coaches_df)
    for i in tqdm(range(0,1000)):
        urlId=str(i)
        dfs=get_team_home_page_dfs_from_cfbstats(i,year=2019)
        if len(dfs)==0: continue
        found_count+=1
        tname = dfs[0].columns[1]
        tname = translate_team_name(tname)
        team = Team.objects.filter(school__name=tname).first()

        cfbs_id,c = CfbstatsUrlId.objects.update_or_create(
            team=team,urlId=urlId,
            defaults=dict(team=team,urlId=urlId)
        )
#main()


## ----- Get 2019 Season Stats -----

In [16]:
def get_home_page_info(teamseason):
    dfs = get_team_home_page_dfs_from_cfbstats(teamseason.team.cfbs.urlId,year=teamseason.season.year)
    team_stats_df = dfs[0]
    team_stats_df = team_stats_df.rename(columns={'Unnamed: 0':'stat_str'})
    tname = team_stats_df.columns[1]
    team_stats_df = team_stats_df.rename(columns={tname:'value'})

    game_result_tdf = dfs[1]
    record_df = dfs[2]

    def get_loc(val):
        if val[0]=='@':
            loc = 'A' 
        elif val[0]=='+':
            loc = 'N'
        else:
            loc = 'H'
        return loc

    game_result_df = game_result_tdf.iloc[:-1,:].copy()
    game_result_df['loc']=game_result_df['Opponent'].map(get_loc)
    game_result_df['Opponent']=game_result_df['Opponent'].str.replace('@','').str.replace('+','')
    game_result_df['Date']=pd.to_datetime(game_result_df['Date'])
    game_result_df=game_result_df.rename(columns={'Game Time':'Game_Time'})

    for tup in record_df.itertuples():
        desc=tup.Split
        record = tup.Record.strip().split('-')
        win=record[0]
        loss=record[1]
        
        ts_record,c = TeamSeasonRecord.objects.update_or_create(
            teamseason=teamseason,desc=desc,
            defaults=dict(teamseason=teamseason,desc=desc,win=win,loss=loss)
        )
    for tup in game_result_df.itertuples():
        date = tup.Date
        attendance = tup.Attendance
        result_str = tup.Result
        result = result_str[0]
        if result not in ['L','W']: result = 'T'
        score_str = result_str[2:]
        score1 = score_str.split('-')[0]
        score2 = score_str.split('-')[1]
        score_arr = [int(score1),int(score2)]
        if result=='W':
            score,oscore=(max(score_arr),min(score_arr))
        else:
            score,oscore=(min(score_arr),max(score_arr))

        oname=tup.Opponent.strip()\
            .replace('St.','State').replace('Ala.','Alabama')
        if oname[0].isnumeric():
            orank = oname.split(' ')[0]
            oname = " ".join(oname.split(' ')[1:])
        else:
            orank=None
        oname = translate_team_name(oname)       

        glen = str(tup.Game_Time)
        glen_arr = glen.split(':')
        hours = float(glen_arr[0])
        hour_frac = float(glen_arr[1])/60 if len(glen.split(':'))>1 else 0
        glen = hours + hour_frac
        opponent = TeamSeason.objects.filter(
            season=teamseason.season,
            team__school__name=oname
        ).first()

        gr,c = GameResult.objects.update_or_create(
            teamseason=teamseason,
            oname=oname,
            date=date,
            defaults=dict(
                teamseason=teamseason,
                opponent=opponent,
                oname=oname,
                date=date,
                score=score,
                oscore=oscore,
                result=result,
                attendance=attendance,
                length=glen,
            )
        )
    for tup in team_stats_df.itertuples():
        stat_str = tup.stat_str
        category=(stat_str.split(':')[0]).replace('/','per')
        desc_str=(stat_str.split(':')[1]).strip() if len(stat_str.split(':'))>1 else category


        desc_arr = desc_str.replace('/','per').split(' - ')
        value_arr = tup.value.replace('%','').split(' - ')
        ovalue_arr = tup.Opponents.replace('%','').split(' - ')


        vtups = list(zip(desc_arr,value_arr,ovalue_arr))
        for vtup in vtups:
            desc=vtup[0]
            value=vtup[1]
            ovalue=vtup[2]
            value = value.replace(',','')
            ovalue = ovalue.replace(',','')  
            if ':' in value:
                value = float(value.split(':')[0]) + float(value.split(':')[1])/60
                ovalue = float(ovalue.split(':')[0]) + float(ovalue.split(':')[1])/60

            if value=='-':value=np.nan
            if ovalue=='-':ovalue=np.nan
            tss,c = TeamSeasonStat.objects.update_or_create(
                teamseason=teamseason,
                category=category,
                desc=desc,
                defaults=dict(
                    teamseason=teamseason,
                    category=category,
                    desc=desc,
                    value=value,
                    ovalue=ovalue               
                )
            )

def get_all_seasons_stats():
    ### Get all Seasons Stats
    for year_num in tqdm(range(2010,2020)):
        season = Season.objects.get(year=year_num)
        print(season)
        for teamseason in tqdm(TeamSeason.objects.filter(season=season)):
            try:
                get_home_page_info(teamseason)
            except IndexError:
                pass
#get_all_seasons_stats()   

## Get Stadium Capacities

In [34]:
def get_stadium_capacities(show_results=True):
    url=r'https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_FBS_football_stadiums'
    df=pd.read_html(url)[0]
    df=df.rename(columns={'Record1':'att_record','Expanded 2':'Expanded'})
    df=df.drop('Image',axis=1)
    def remove_wiki_citation(df,col_list=None):
        if col_list == None:
            col_list = df.columns
        for col in col_list:
            df[col].str.split('[',expand=True)[0]

    DFC = DataFrameCleaner(df)
    DFC.remove_wiki_citation()
    DFC.keep_numeric_only(col_list=['Capacity','Built','Expanded'],type_coerce=np.float)
    df = DFC.df
    df=df.replace(np.nan,None)
    df['Built']=df['Built'].astype(str).str[:4].astype(int)
    df['Expanded']=df['Expanded'].astype(str).str[:4].astype(int)
    for tup in df.itertuples():
        if tup.Capacity > 200_000:
            cap=float(str(tup.Capacity)[:5])
        else:
            cap = tup.Capacity
        stadium,c = Stadium.objects.update_or_create(
            name=tup.Stadium,
            defaults=dict(
                name=tup.Stadium,
                city=tup.City,
                state=tup.State,
                year_built=tup.Built,
                year_expanded=tup.Expanded,
                capacity=cap,
                wiki_team_name=tup.Team,
                surface=tup.Surface
            )
        )

        school_name = stadium.wiki_team_name
        if stadium.wiki_team_name == 'Miami':
            school_name = 'Miami (Fla.)' if tup.State=='OH' else 'Miami (Ohio)'
        if stadium.wiki_team_name == 'Louisiana':
            school_name = 'Louisiana-Lafayette'
        if stadium.wiki_team_name[:5] == 'Hawai':
            school_name = 'Hawaii'  

        team = Team.objects.filter(school__name=school_name).first()
        if pd.isna(team):
            school_name = team_match_dict.get(school_name)
            team = Team.objects.filter(school__name=school_name).first()

        teamseason_qs = team.teamseason_set.filter(season__year__gte=stadium.year_built)
        if len(teamseason_qs)==0:
            print(stadium)
            print(stadium.year_built)
            print(team)
        if show_results:
            print(stadium)
get_stadium_capacities(show_results=True) 

Aggie Memorial Stadium, Las Cruces, NM
Alamodome, San Antonio, TX
Alaska Airlines Field at Husky Stadium, Seattle, WA
Albertsons Stadium, Boise, ID
Allen E. Paulson Stadium, Statesboro, GA
Aloha Stadium, Honolulu, HI
Alumni Stadium, Chestnut Hill, MA
Amon G. Carter Stadium, Fort Worth, TX
Apogee Stadium, Denton, TX
Arizona Stadium, Tucson, AZ
Arthur L. Williams Stadium, Lynchburg, VA
Autzen Stadium, Eugene, OR
Bagwell Field at Dowdy–Ficklen Stadium, Greenville, NC
BB&T Field, Winston-Salem, NC
Beaver Stadium, University Park, PA
Benson Field at Yulman Stadium, New Orleans, LA
Bill Snyder Family Football Stadium, Manhattan, KS
Blaik Field at Michie Stadium, West Point, NY
Bobby Bowden Field at Doak S. Campbell Stadium, Tallahassee, FL
Bobby Dodd Stadium at Historic Grant Field, Atlanta, GA
Boone Pickens Stadium, Stillwater, OK
Brigham Field at Huskie Stadium, DeKalb, IL
Brooks Field at Wallace Wade Stadium, Durham, NC
Brooks Stadium, Conway, SC
Bryant–Denny Stadium, Tuscaloosa, AL
Bulld

## Get Grad Rates



In [14]:
def get_grad_rates():
    path = join(pwd,f'z_data\Graduation_Success_Rate.html')
    df=pd.read_html(path)[0]
    df['sog']=df['School'].copy()
    display(df)
    def match_teams(name):
        if name in team_match_dict.keys():
            return team_match_dict[name]
        else:
            return name
    df['School']=df['School'].map(match_teams)
    df['School']=df['School'].str.replace('University of','').str.strip()
    df['year']=df['Cohort Year']
    display(df.sample(5).head())
    df['School']=df['School'].str.replace('University of','').str.strip()
    df['School']=df['School'].str.replace('University','').str.strip()
    for tup in df.itertuples():
        season = Season.objects.get(year=tup.year)
        team = Team.objects.filter(school__name=tup.School).first()
        if pd.isna(team): continue
        ts,c = TeamSeason.objects.update_or_create(
            team=team,season=season,
            defaults=dict(gsr=tup.GSR,fgr=tup.FGR)
        )
        print(ts.GSR)
#get_grad_rates()

,Cohort Year,School,Conference,Sport,State,GSR,FGR,GSR Report,FGR Report,sog
0,1998,University of Akron,Mid-American Conference,Football,OH,60.0,53.0,NaN,NaN,University of Akron
1,1998,Alabama State University,Southwestern Athletic Conf.,Football,AL,44.0,41.0,NaN,NaN,Alabama State University
2,1998,University of Alabama,Southeastern Conference,Football,AL,39.0,39.0,NaN,NaN,University of Alabama
3,1998,University of Alabama at Birmingham,Conference USA,Football,AL,51.0,52.0,NaN,NaN,University of Alabama at Birmingham
4,1998,Alcorn State University,Southwestern Athletic Conf.,Football,MS,88.0,45.0,NaN,NaN,Alcorn State University
...,...,...,...,...,...,...,...,...,...,...
3584,2012,Presbyterian College,Big South Conference,Football,SC,72.0,56.0,NaN,NaN,Presbyterian College
3585,2012,1 - DI CA Test,Independent,Football,AK,100.0,100.0,NaN,NaN,1 - DI CA Test
3586,2012,"University of Arkansas, Pine Bluff",Southwestern Athletic Conf.,Football,AR,66.0,51.0,NaN,NaN,"University of Arkansas, Pine Bluff"
3587,2012,University of the Incarnate Word,Southland Conference,Football,TX,86.0,56.0,NaN,NaN,University of the Incarnate Word


,Cohort Year,School,Conference,Sport,State,GSR,FGR,GSR Report,FGR Report,sog,year
376,1999,South Carolina State University,Mid-Eastern Athletic Conf.,Football,SC,60.0,42.0,NaN,NaN,South Carolina State University,1999
754,2001,Kansas State,Big 12 Conference,Football,KS,67.0,70.0,NaN,NaN,Kansas State University,2001
3126,2011,The Citadel,Southern Conference,Football,SC,82.0,67.0,NaN,NaN,The Citadel,2011
494,2000,Florida State,Atlantic Coast Conference,Football,FL,58.0,48.0,NaN,NaN,Florida State University,2000
1199,2003,Florida State,Atlantic Coast Conference,Football,FL,64.0,58.0,NaN,NaN,Florida State University,2003


1998, Football, Akron
1998, Football, Alabama
1998, Football, Alabama at Birmingham
1998, Football, Appalachian State
1998, Football, Arizona State
1998, Football, Arizona
1998, Football, Arkansas State
1998, Football, Arkansas
1998, Football, Auburn
1998, Football, Ball State
1998, Football, Baylor
1998, Football, Boise State
1998, Football, Boston College
1998, Football, Bowling Green
1998, Football, Brigham Young
1998, Football, Buffalo
1998, Football, Fresno State
1998, Football, California
1998, Football, UCLA
1998, Football, Central Florida
1998, Football, Central Michigan
1998, Football, Cincinnati
1998, Football, Clemson
1998, Football, Colorado State
1998, Football, Colorado
1998, Football, Connecticut
1998, Football, Duke
1998, Football, East Carolina
1998, Football, Eastern Michigan
1998, Football, Florida State
1998, Football, Florida
1998, Football, Georgia Southern
1998, Football, Georgia Tech
1998, Football, Georgia
1998, Football, Hawaii
1998, Football, Houston
1998, Fo

2000, Football, Louisiana-Monroe
2000, Football, Northern Illinois
2000, Football, Northwestern
2000, Football, Notre Dame
2000, Football, Ohio State
2000, Football, Ohio
2000, Football, Oklahoma State
2000, Football, Oklahoma
2000, Football, Oregon State
2000, Football, Oregon
2000, Football, Penn State
2000, Football, Pittsburgh
2000, Football, Purdue
2000, Football, Rice
2000, Football, Rutgers
2000, Football, San Diego State
2000, Football, San Jose State
2000, Football, South Carolina
2000, Football, South Florida
2000, Football, Southern California
2000, Football, Southern Methodist
2000, Football, Southern Mississippi
2000, Football, Texas State
2000, Football, Louisiana-Lafayette
2000, Football, Stanford
2000, Football, Syracuse
2000, Football, Temple
2000, Football, Tennessee
2000, Football, Texas A&M
2000, Football, Texas Christian
2000, Football, Texas Tech
2000, Football, Texas
2000, Football, Texas-El Paso
2000, Football, Toledo
2000, Football, Troy
2000, Football, Tulane


2002, Football, Western Michigan
2002, Football, Wisconsin
2002, Football, Wyoming
2003, Football, Akron
2003, Football, Alabama
2003, Football, Alabama at Birmingham
2003, Football, Appalachian State
2003, Football, Arizona State
2003, Football, Arizona
2003, Football, Arkansas State
2003, Football, Arkansas
2003, Football, Auburn
2003, Football, Ball State
2003, Football, Baylor
2003, Football, Boise State
2003, Football, Boston College
2003, Football, Bowling Green
2003, Football, Brigham Young
2003, Football, Buffalo
2003, Football, Fresno State
2003, Football, California
2003, Football, UCLA
2003, Football, Central Florida
2003, Football, Central Michigan
2003, Football, Cincinnati
2003, Football, Clemson
2003, Football, Coastal Carolina
2003, Football, Colorado State
2003, Football, Colorado
2003, Football, Connecticut
2003, Football, Duke
2003, Football, East Carolina
2003, Football, Eastern Michigan
2003, Football, Florida Atlantic
2003, Football, Florida International
2003, Fo

2005, Football, Louisville
2005, Football, Marshall
2005, Football, Maryland
2005, Football, Massachusetts
2005, Football, Memphis
2005, Football, Miami (Ohio)
2005, Football, Miami (Fla.)
2005, Football, Michigan State
2005, Football, Michigan
2005, Football, Middle Tennessee
2005, Football, Minnesota
2005, Football, Mississippi State
2005, Football, Mississippi
2005, Football, Missouri
2005, Football, North Carolina
2005, Football, Nebraska
2005, Football, Nevada-Las Vegas
2005, Football, Nevada
2005, Football, New Mexico State
2005, Football, New Mexico
2005, Football, North Carolina State
2005, Football, North Texas
2005, Football, Louisiana-Monroe
2005, Football, Northern Illinois
2005, Football, Northwestern
2005, Football, Notre Dame
2005, Football, Ohio State
2005, Football, Ohio
2005, Football, Oklahoma State
2005, Football, Oklahoma
2005, Football, Oregon State
2005, Football, Oregon
2005, Football, Penn State
2005, Football, Pittsburgh
2005, Football, Purdue
2005, Football, 

2007, Football, Texas Tech
2007, Football, Texas
2007, Football, Texas-El Paso
2007, Football, Texas-San Antonio
2007, Football, Toledo
2007, Football, Troy
2007, Football, Tulane
2007, Football, Tulsa
2007, Football, Air Force
2007, Football, Army
2007, Football, Navy
2007, Football, Utah State
2007, Football, Utah
2007, Football, Vanderbilt
2007, Football, Virginia Tech
2007, Football, Virginia
2007, Football, Wake Forest
2007, Football, Washington State
2007, Football, Washington
2007, Football, West Virginia
2007, Football, Western Kentucky
2007, Football, Western Michigan
2007, Football, Wisconsin
2007, Football, Wyoming
2008, Football, Akron
2008, Football, Alabama
2008, Football, Alabama at Birmingham
2008, Football, Appalachian State
2008, Football, Arizona State
2008, Football, Arizona
2008, Football, Arkansas State
2008, Football, Arkansas
2008, Football, Auburn
2008, Football, Ball State
2008, Football, Baylor
2008, Football, Boise State
2008, Football, Boston College
2008, 

2010, Football, Cincinnati
2010, Football, Clemson
2010, Football, Coastal Carolina
2010, Football, Colorado State
2010, Football, Colorado
2010, Football, Connecticut
2010, Football, Duke
2010, Football, East Carolina
2010, Football, Eastern Michigan
2010, Football, Florida Atlantic
2010, Football, Florida International
2010, Football, Florida State
2010, Football, Florida
2010, Football, Georgia Southern
2010, Football, Georgia State
2010, Football, Georgia Tech
2010, Football, Georgia
2010, Football, Hawaii
2010, Football, Houston
2010, Football, Illinois
2010, Football, Indiana
2010, Football, Iowa State
2010, Football, Iowa
2010, Football, Kansas State
2010, Football, Kansas
2010, Football, Kent State
2010, Football, Kentucky
2010, Football, Liberty
2010, Football, LSU
2010, Football, Louisiana Tech
2010, Football, Louisville
2010, Football, Marshall
2010, Football, Maryland
2010, Football, Massachusetts
2010, Football, Memphis
2010, Football, Miami (Ohio)
2010, Football, Miami (F

2012, Football, Michigan State
2012, Football, Michigan
2012, Football, Middle Tennessee
2012, Football, Minnesota
2012, Football, Mississippi State
2012, Football, Mississippi
2012, Football, Missouri
2012, Football, North Carolina
2012, Football, Charlotte
2012, Football, Nebraska
2012, Football, Nevada-Las Vegas
2012, Football, Nevada
2012, Football, New Mexico State
2012, Football, New Mexico
2012, Football, North Carolina State
2012, Football, North Texas
2012, Football, Louisiana-Monroe
2012, Football, Northern Illinois
2012, Football, Northwestern
2012, Football, Notre Dame
2012, Football, Ohio State
2012, Football, Ohio
2012, Football, Oklahoma State
2012, Football, Oklahoma
2012, Football, Old Dominion
2012, Football, Oregon State
2012, Football, Oregon
2012, Football, Penn State
2012, Football, Pittsburgh
2012, Football, Purdue
2012, Football, Rice
2012, Football, Rutgers
2012, Football, San Diego State
2012, Football, San Jose State
2012, Football, South Alabama
2012, Footba